# 第5章 顧客の退会を予測する10本ノック (41~50)

In [1]:
# 下準備として，データのあるディレクトリに移動しておく
import os

DATA_ROOT_DIR = "./sample/5章/"
os.chdir(DATA_ROOT_DIR)

## ノック41 データを読み込んで利用データを成形しよう
今回は，データを読み込むだけでなく，データの成形も行う．

In [2]:
import pandas as pd
customer = pd.read_csv("customer_join.csv")
use_log_months = pd.read_csv("use_log_months.csv")

In [3]:
year_months = list(use_log_months["年月"].unique())
use_log = pd.DataFrame()

for i in range(1, len(year_months)):
    tmp = use_log_months.loc[use_log_months["年月"] == year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = use_log_months.loc[use_log_months["年月"] == year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    use_log = pd.concat([use_log, tmp], ignore_index=True)

use_log.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
